In [1]:
import pyomo.environ as pyo
import numpy as np
import pandas as pd

In [2]:
class SolveAction():
    def __init__(self, model, solver_name):
        self.model = model
        self.solver_name = solver_name

    def solve(self):
        solver_name = self.solver_name
        model = self.model
        
        solver = pyo.SolverFactory(solver_name)
        instance = model.create_instance()
        result = solver.solve(instance)
        instance.A.display()

In [3]:
class CalculateUsefulFilesAction():
    def __init__(self, useful_requirements, T, D):
        self.useful_requirements = useful_requirements
        self.T = T
        self.D = D

    def calculate(self):
        useful_requirements = self.useful_requirements
        T = self.T
        calculate_dependencies = self.__calculate_dependencies

        useful_files = np.dot(useful_requirements, T)
        m = len(useful_files)
        for i in range(m):
            dependencies = calculate_dependencies(i)
            useful_files += dependencies
        return useful_files

    def __calculate_dependencies(self, i):
        useful_requirements = self.useful_requirements
        T = self.T
        D = self.D
        calculate_dependencies = self.__calculate_dependencies
        
        if i == 0:
            useful_files = np.dot(useful_requirements, T)
            return np.dot(useful_files, D)
        else:
            dependencies = calculate_dependencies(i - 1)
            return np.dot(dependencies, D)

In [4]:
class Checker():
    def __init__(self, constraints, varlist):
        self.constraints = constraints
        self.varlist = varlist

    def check(self, vector, check_function_1, check_function_2):
        constraints = self.constraints
        varlist = self.varlist
        
        result = []
        for x in vector:
            f = varlist.add()
            result.append(f)
            constraint_1 = check_function_1(x, f)
            constraint_2 = check_function_2(x, f)
            constraints.add(constraint_1)
            constraints.add(constraint_2)
        return np.array(result)

In [5]:
class IncludeChecker():
    def __init__(self, checker, M):
        self.checker = checker
        self.M = M

    def check(self, vector):
        checker = self.checker
        M = self.M
        
        check_include_1 = lambda x, f: (f + 1 / M <= x + 1)
        check_include_2 = lambda x, f: (x <= M * f)

        result = checker.check(vector, check_include_1, check_include_2)
        return result

In [6]:
class ImplementChecker():
    def __init__(self, checker, M):
        self.checker = checker
        self.M = M

    def check(self, vector):
        checker = self.checker
        M = self.M

        check_implemented_1 = lambda x, f: (x >= f)
        check_implemented_2 = lambda x, f: (x + 1 / M <= 1 + M * f)

        result = checker.check(vector, check_implemented_1, check_implemented_2)
        return result

In [7]:
class CalculatePluginsAction():
    def __init__(self, calculate_useful_files_action, include_checker, A):
        self.calculate_useful_files_action = calculate_useful_files_action
        self.include_checker = include_checker
        self.A = A

    def calculate(self):
        calculate_useful_files_action = self.calculate_useful_files_action
        include_checker = self.include_checker
        A = self.A

        useful_files = calculate_useful_files_action.calculate()
        plugins = np.dot(useful_files, A)
        result = include_checker.check(plugins)
        return result

In [8]:
class AddMultiplyConstraintsAction():
    def __init__(self, constraints, varlist):
        self.constraints = constraints
        self.varlist = varlist

    def add_multiply_constraints(self, x, y):
        constraints = self.constraints
        varlist = self.varlist
        
        f = varlist.add()
        constraints.add((x + y <= f + 1))
        constraints.add((f <= x))
        constraints.add((f <= y))
        constraints.add((f >= 0))
        return f

In [9]:
class CalculateDeliveryFilesAction():
    def __init__(self, calculate_plugins_action, add_multiply_constraints_action, A):
        self.calculate_plugins_action = calculate_plugins_action
        self.add_multiply_constraints_action = add_multiply_constraints_action
        self.A = A

    def calculate(self):
        calculate_plugins_action = self.calculate_plugins_action
        A = self.A
        bin_multiply = self.__bin_multiply
        
        plugins = calculate_plugins_action.calculate()
        result = bin_multiply(A, plugins)
        return result

    def __bin_multiply(self, matrix, vector):
        add_multiply_constraints_action = self.add_multiply_constraints_action
        
        (rows_count, cols_count) = np.shape(matrix)
        result = []
        for row_number in range(rows_count):
            matrix_row = matrix[row_number]
            terms = []
            for col_number in range(cols_count):
                vector_element = vector[col_number]
                matrix_element = matrix_row[col_number]
                term = add_multiply_constraints_action.add_multiply_constraints(vector_element, matrix_element)
                terms.append(term)
            sum_terms = sum(terms)
            result.append(sum_terms)
        return np.array(result)

In [10]:
class CalculateDeliveryRequirementsAction():
    def __init__(self, calculate_delivery_files_action, implement_checker, T):
        self.calculate_delivery_files_action = calculate_delivery_files_action
        self.implement_checker = implement_checker
        self.T = T

    def calculate(self):
        calculate_delivery_files_action = self.calculate_delivery_files_action
        implement_checker = self.implement_checker
        T = self.T

        delivery_files = calculate_delivery_files_action.calculate()
        delivery_requirements = np.dot(T, delivery_files)
        result = implement_checker.check(delivery_requirements)
        return result

In [11]:
class CalculateEquipmentCostAction():
    def __init__(self, calculate_delivery_requirements_action, P):
        self.calculate_delivery_requirements_action = calculate_delivery_requirements_action
        self.P = P

    def calculate(self):
        calculate_delivery_requirements_action = self.calculate_delivery_requirements_action
        P = self.P

        delivery_requirements = calculate_delivery_requirements_action.calculate()
        costs = np.dot(P, delivery_requirements)
        result = sum(costs)
        return result

In [12]:
class ModelBuilder():

    def __init__(self, M, k, T, D, P, E):
        self.M = M
        self.k = k
        self.T = T
        self.D = D
        self.P = P
        self.E = E
    
    def build(self):
        M = self.M
        k = self.k
        
        T = self.T
        D = self.D
        P = self.P
        E = self.E

        m = np.shape(T)[1]
        
        model = pyo.ConcreteModel(name = 'Optimal decomposition')
        model.constraints = pyo.ConstraintList()
        model.f = pyo.VarList(domain=pyo.Binary)
        set_m = pyo.Set(initialize=range(m))
        set_k = pyo.Set(initialize=range(k))
        model.A = pyo.Var(set_m, set_k, domain=pyo.Binary)
        A = np.array(model.A)

        for row in A:
            model.constraints.add((sum(row) == 1))

        equipment_costs = []
        for useful_requirements in E:
            calculate_useful_files_action = CalculateUsefulFilesAction(useful_requirements, T, D)
            checker = Checker(model.constraints, model.f)
            include_checker = IncludeChecker(checker, M)
            calculate_plugins_action = CalculatePluginsAction(calculate_useful_files_action, include_checker, A)
            add_multiply_constraints_action = AddMultiplyConstraintsAction(model.constraints, model.f)
            calculate_delivery_files_action = CalculateDeliveryFilesAction(calculate_plugins_action, add_multiply_constraints_action, A)
            implement_checker = ImplementChecker(checker, M)
            calculate_delivery_requirements_action = CalculateDeliveryRequirementsAction(calculate_delivery_files_action, implement_checker, T)
            calculate_equipment_cost_action = CalculateEquipmentCostAction(calculate_delivery_requirements_action, P)
            
            equipment_cost = calculate_equipment_cost_action.calculate()
            equipment_costs.append(equipment_cost)
            
        model.OBJ = pyo.Objective(expr = sum(equipment_costs), sense=pyo.minimize)
        return model

In [13]:
M = 10 ** 6

In [14]:
k = 3

In [15]:
T = np.array([
    [  1,   0,   0,   0],  # Требование № 1 реализовано в файле № 1
    [0.5, 0.5,   0,   0],  # Требование № 2 реализовано в файлах № 1 и 2
    [  0, 0.5, 0.5,   0],  # Требования № 3 реализовано в файлах № 2 и 3
    [  0,   0, 0.5, 0.5],  # Требование № 4 реализовано в файлах № 3 и 4
    [  0,   0,   0,   1]   # Требовнаие № 5 реализовано в файле № 4
])

In [16]:
D = np.array([
    [0, 1, 0, 0],  # Файл № 1 имеет зависимость на файл № 2
    [0, 0, 0, 0],  # Файл № 2 не имеет зависимостей на другие файлы
    [0, 0, 0, 0],  # Файл № 3 не имеет зависимостей на другие файлы
    [0, 0, 0, 0]   # Файл № 4 не имеет зависимостей на другие файлы
]) 

In [17]:
P = np.array([
    [1, 0, 0, 0, 0], # Стоимость сопровождения функционала, если в поставку войдет требование № 1
    [0, 1, 0, 0, 0], # Стоимость сопровождения функционала, если в поставку войдет требование № 2
    [0, 0, 1, 0, 0], # Стоимость сопровождения функционала, если в поставку войдет требование № 3
    [0, 0, 0, 1, 0], # Стоимость сопровождения функционала, если в поставку войдет требование № 4
    [0, 0, 0, 0, 1]  # Стоимость сопровождения функционала, если в поставку войдет требование № 5
])

In [18]:
E = np.array([
    [1, 0, 0, 0, 0]
])

In [19]:
model_builder = ModelBuilder(M, k, T, D, P, E)
model = model_builder.build()
model.pprint()

2 Var Declarations
    A : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :     0 :  None :     1 : False :  True : Binary
        (0, 1) :     0 :  None :     1 : False :  True : Binary
        (0, 2) :     0 :  None :     1 : False :  True : Binary
        (1, 0) :     0 :  None :     1 : False :  True : Binary
        (1, 1) :     0 :  None :     1 : False :  True : Binary
        (1, 2) :     0 :  None :     1 : False :  True : Binary
        (2, 0) :     0 :  None :     1 : False :  True : Binary
        (2, 1) :     0 :  None :     1 : False :  True : Binary
        (2, 2) :     0 :  None :     1 : False :  True : Binary
        (3, 0) :     0 :  None :     1 : False :  True : Binary
        (3, 1) :     0 :  None :     1 : False :  True : Binary
        (3, 2) :     0 :  None :     1 : False :  True : Binary
    f : Size=20, Index={1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
        

/tmp/ipykernel_2530/3597262246.py:28: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  A = np.array(model.A)


In [22]:
with_glpk = SolveAction(model, 'glpk')
with_glpk.solve()

A : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (0, 0) :     0 :   1.0 :     1 : False : False : Binary
    (0, 1) :     0 :   0.0 :     1 : False : False : Binary
    (0, 2) :     0 :   0.0 :     1 : False : False : Binary
    (1, 0) :     0 :   0.0 :     1 : False : False : Binary
    (1, 1) :     0 :   0.0 :     1 : False : False : Binary
    (1, 2) :     0 :   1.0 :     1 : False : False : Binary
    (2, 0) :     0 :   0.0 :     1 : False : False : Binary
    (2, 1) :     0 :   1.0 :     1 : False : False : Binary
    (2, 2) :     0 :   0.0 :     1 : False : False : Binary
    (3, 0) :     0 :   0.0 :     1 : False : False : Binary
    (3, 1) :     0 :   1.0 :     1 : False : False : Binary
    (3, 2) :     0 :   0.0 :     1 : False : False : Binary
